# Classificador Home e Bart - Rede Neural Densa Simples

A base de dados de imagens de treinamento e teste estão no arquivo personagens.csv da raiz

## Importando bibliotecas

In [4]:
!pip install -q tensorflow==2.16.1

In [5]:
# Importacao desta lib para desativar erro no TensorFlow
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [8]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

In [9]:
tf.__version__, np.__version__, pd.__version__

('2.16.1', '1.26.4', '2.2.2')

## Carregar o dataset

In [10]:
df = pd.read_csv("personagens.csv")

# Separar características (X) e classe (y)
X = df.iloc[:, :-1].values  # Todas as colunas menos a última (características)
y = df.iloc[:, -1].values   # Última coluna (classe)

## Codificar as classes

In [11]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # Converter 'Homer' e 'Bart' para números
y_categorical = to_categorical(y_encoded)  # Converter para one-hot encoding

## Dividir a base em treino e teste

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.25, random_state=42)

## Normalizar os dados

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Construir a rede neural densa

In [14]:
model = Sequential([
    Dense(16, input_shape=(X_train.shape[1],), activation="relu"),  # Camada escondida com 16 neurônios
    Dense(8, activation="relu"),  # Camada escondida com 8 neurônios
    Dense(2, activation="softmax")  # Camada de saída para 2 classes (Homer e Bart)
])

# Compilar o modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

/home/lucas/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266 (1.04 KB)

 Trainable params: 266 (1.04 KB)

 Non-trainable params: 0 (0.00 B)

## Treinar o modelo

In [16]:
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.3769 - loss: 0.7186 - val_accuracy: 0.3636 - val_loss: 0.7130
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3938 - loss: 0.7128 - val_accuracy: 0.5682 - val_loss: 0.6796
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5518 - loss: 0.6358 - val_accuracy: 0.7727 - val_loss: 0.6508
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7168 - loss: 0.6165 - val_accuracy: 0.8182 - val_loss: 0.6197
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7717 - loss: 0.5891 - val_accuracy: 0.8182 - val_loss: 0.5887
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8508 - loss: 0.5510 - val_accuracy: 0.8636 - val_loss: 0.5580
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7932 - loss: 0.5419 - val_accuracy: 0.9091 - val_loss: 0.5286
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8484 - loss: 0.5057 - val_accuracy: 0.9091 - val_loss

## Avaliar o modelo

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {accuracy:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9524 - loss: 0.1914
Acurácia no conjunto de teste: 0.96


## Prever em novos dados (opcional)

In [18]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(y_test, axis=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [19]:
# Exibir os primeiros 10 resultados
for i in range(10):
    print(f"Imagem {i+1}: Classe Prevista: {encoder.inverse_transform([predicted_classes[i]])[0]}, Classe Real: {encoder.inverse_transform([actual_classes[i]])[0]}")

Imagem 1: Classe Prevista: Bart, Classe Real: Bart
Imagem 2: Classe Prevista: Homer, Classe Real: Homer
Imagem 3: Classe Prevista: Bart, Classe Real: Bart
Imagem 4: Classe Prevista: Homer, Classe Real: Homer
Imagem 5: Classe Prevista: Bart, Classe Real: Bart
Imagem 6: Classe Prevista: Bart, Classe Real: Bart
Imagem 7: Classe Prevista: Homer, Classe Real: Homer
Imagem 8: Classe Prevista: Bart, Classe Real: Bart
Imagem 9: Classe Prevista: Homer, Classe Real: Bart
Imagem 10: Classe Prevista: Bart, Classe Real: Bart
